In [1]:
import os
import sys
from pathlib import Path
import pandas as pd

%matplotlib inline

In [2]:
sys.path.append(r"../LUCinSA_helpers")
from rf import *

In [3]:
'''
PARAMETERS: modify in notebook_params notebook, then run that notebook and this cell to update here
DO NOT modify this cell
'''

%store -r basic_config
print("Basic Parameters: \n time-series data is in (smooth_dir): {} \n"
      " modelling year is (filter_year param): {} (this is first year if season spans two years)"
      .format(basic_config['smooth_dir'], basic_config['filter_yr']))
%store -r classification_params

print("Classification_Params: \n" 
      " modelling mode is {} \n"
      " model_type = {} \n"
      " output files are saved to (model_dir): {} \n" 
      " shared input files are in (main_model_dir): {} \n"
      " sample_model = {} \n feature_model = {} \n model_name = {} \n"
      " the full sample pt file: {} \n"
      " the full sample dataframe with the feature model applied: {} \n"
      " the subset pt file based on the sample model: {} \n"
      " lc_class = {} \n ranhold = {} \n impmeth = {}"
      .format(classification_params['model_mode'],classification_params['model_type'],classification_params['model_dir'],
              classification_params['main_model_dir'],classification_params['sample_model'],classification_params['feature_model'],
              classification_params['model_name'],basic_config['ptfile'],classification_params['samp_pix_vars'],classification_params['samp_pts'],
              classification_params['lc_mod'],classification_params['ranhold'],classification_params['impmeth']))
    

Basic Parameters: 
 time-series data is in (smooth_dir): /home/downspout-cel/paraguay_lc/stac/grids 
 modelling year is (filter_year param): 2021 (this is first year if season spans two years)
Classification_Params: 
 modelling mode is testing 
 model_type = RF 
 output files are saved to (model_dir): /home/downspout-cel/paraguay_lc/vector/tests_KW 
 shared input files are in (main_model_dir): /home/downspout-cel/paraguay_lc/classification/RF 
 sample_model = bal400mix1 
 feature_model = Max 
 model_name = Max_bal400mix1 
 the full sample pt file: /home/downspout-cel/paraguay_lc/vector/sampleData/SamplePts_Dec2023_ALL.csv 
 the full sample dataframe with the feature model applied: /home/downspout-cel/paraguay_lc/vector/ptsgdb_Max.csv 
 the subset pt file based on the sample model: /home/downspout-cel/paraguay_lc/classification/RF/sample_dfs/bal400mix1.csv 
 lc_class = all 
 ranhold = 29 
 impmeth = Impurity


#### Set new variables here for temp model testing: -- SKIP if keeping original model

In [5]:
## Set new variables here for temp model testing:
feature_model = "Max"
## Sample model options currently: bal400mix1 | bal400mix2 | bal400mix3
sample_model = "bal400mix3"

## The following will set themselves based on the above variables:
classification_params['feature_model'] = feature_model
classification_params['sample_model'] = sample_model
classification_params['model_name'] = '{}_{}'.format(feature_model, sample_model)
classification_params['samp_pix_vars'] = '{}/ptsgdb_{}.csv'.format(classification_params['model_dir'],feature_model)
classification_params["samp_pts"] = '/home/downspout-cel/paraguay_lc/classification/RF/sample_dfs/{}.csv'.format(sample_model)
print('Now working with sample_model: {} \n New output model will be named: {}'
      .format(classification_params['sample_model'],classification_params['model_name']))

Now working with sample_model: bal400mix3 
 New output model will be named: Max_bal400mix3


## Merge dataframes together

In [6]:
lut=pd.read_csv('../Class_LUT.csv')
#print(lut.sort_values('LC_UNQ')[['LC_UNQ','USE_NAME','LC25','LC25_name']])

samp_pts = pd.read_csv(classification_params['samp_pts'])
print(samp_pts.columns.tolist())
#if mod_name == "base1000":
#    samp_pts.rename(columns = {"Unnamed: 0": 'OID_'}, inplace = True)

pix_vars = pd.read_csv(classification_params['samp_pix_vars'])
#print(pix_vars.columns.tolist())

pix_data = samp_pts.merge(pix_vars, left_on='OID_', right_on='OID_', how='inner')
print('sample breakdown by LC25 class:')
print(pix_data['LC25_name'].value_counts())

#pixdf = pix_data.merge(lut, left_on='Class', right_on='USE_NAME', how='left')

classification_params["pixdf"] = pix_data
#print(classification_params['pixdf'].columns.tolist())

['Unnamed: 0', 'OID_', 'UNQ8858', 'Class', 'LC_UNQ', 'NOTES', 'YCoord', 'XCoord', 'SampMethod', 'PurePixel', 'FieldWidth', 'PureCrop', 'Date', 'Photo', 'surface', 'rand', 'TESTSET20', 'TESTSET10', 'UNQ8858.1', 'rand2', 'estrat', 'smlhld_1ha', 'smlhd_halfha', 'LC_UNQ_y', 'USE_NAME', 'LC25', 'perLC25E', 'LC25_name', 'ratios']
sample breakdown by LC25 class:
Trees-Forest         1711
Grassland-Managed    1597
Crops-Soybeans       1379
Crops-mix            1210
Mixed-VegEdge        1196
Mixed-path           1183
Grassland-Wet        1062
Grassland-Natural     632
Shrub                 494
Trees-disturbed       429
Crops-Yerba-Mate      410
Grass_tree-mix        408
TreePlant             407
NoVeg_Water           401
Crops-Rice            398
NoVeg_Bare            397
Crops-Sugar           369
Crops-Banana          313
Crops-Orchard         289
NoVeg_Built           284
Cleared               266
Crops-Corn            244
Trees-Forest_palm     213
Burnt-woody            74
Name: LC25_name, d

## View the look up table
These are the different LC_models to group things in classification and to translate between numerical map categories and text labels

In [7]:
lut=pd.read_csv('../Class_LUT.csv')
lut.drop(['Description'], axis=1, inplace=True)

print(lut.sort_values('LC_UNQ'))

    LC_UNQ            USE_NAME          Segmentation  LCTrans  \
0        1               NoVeg                 NoVeg        1   
1        2          NoVeg_Bare                   NaN        2   
2        3         NoVeg_Built                   NaN        3   
3        7         NoVeg_Water                   NaN        7   
4        9          Mixed-path                   NaN        9   
5       10             Cleared                   NaN       10   
6       11       TreePlant-new                   NaN       11   
7       12   Grassland-Natural                   NaN       15   
8       13   Grassland-Managed  LowVeg_grass_managed       15   
9       15           Grassland          LowVeg_grass       15   
10      17       Grassland-Wet                   NaN       15   
11      18     Mixed-GrassEdge                   NaN       19   
12      19     Mixed-FieldEdge                   NaN       19   
13      20              LowVeg                LowVeg       20   
14      23  Crops-horticu

## get sample dataframe:
 pixdf is the combination of the sample point file and variable stack for those points (pix_vars).
 This is created in notebooks 6a and 6b.

In [20]:
"""print(classification_params['pixdf'])
pixdf = pd.read_csv(classification_params['pixdf'])
#pixdf = pixdf.dropna(inplace = True)
#print(pixdf)
#print(pixdf.isnull().any().any())
print('sample breakdown by {}:'.format(classification_params['lc_mod']))
label_col, new_lut = get_class_col(classification_params['lc_mod'], lut)
if '{}_name'.format(label_col) in pixdf.columns:
    print(pixdf['{}_name'.format(label_col)].value_counts())
else:
    pixdf2 = pixdf.merge(new_lut[['USE_NAME','{}'.format(label_col),'{}_name'.format(label_col)]], left_on='Class', right_on='USE_NAME', how='left')
    print(pixdf2['{}_name'.format(label_col)].value_counts())""";

## create rf model
this uses the multiclass RandomForestClassifier method from sklearn.ensemble (code is in ../LUCinSA_helpers/rf.py)

To use a different classification model, change 'lc_mod' in the parameters and rerun
current models = ('All' | 'trans_cats" | 'crop_nocrop' | 'crop_nocrop_medcrop' | 'crop_nocrop_medcrop_tree' | 'veg' | 'cropType' or 'single_X' (where X is any unique string in the USE_NAME column) for binary classification of X vs all else) 

In [8]:
rf0 = rf_model(classification_params['pixdf'],
         classification_params['model_dir'],
         classification_params['lc_mod'],
         classification_params['impmeth'],
         classification_params['ranhold'],
         classification_params['model_name'],
         lut)

class_col = LC25
there are 15366 pts in the full data set
there are 15366 sample points after removing those without clear class
There are 12648 training features
Out-of-bag score estimate: 0.662
Mean accuracy score: 0.646
getting confusion matrix based on LC25...
number of holdout pixels = 2718


## view confusion matrices
Note parameters: (pred_col, obs_col, lut, lc_mod_map, lc_mod_acc, print_cm=False, out_dir=None, model_name=None)
To print cm to csv file, change print_cm to True and provide an out_dir and model_Name

In [10]:
out_dir = os.path.join(classification_params['local_dir']/'cms')

cm_cropNoCrop = get_confusion_matrix(rf0[1]['pred'],rf0[1]['label'],lut,classification_params['lc_mod'],'cropNoCrop',True, os.path.join(out_dir,'cropNoCrop_cms'),classification_params['model_name'])
cm_cropType = get_confusion_matrix(rf0[1]['pred'],rf0[1]['label'],lut,classification_params['lc_mod'],'cropType',True,os.path.join(out_dir,'cropType_cms'),classification_params['model_name'])
cm_veg = get_confusion_matrix(rf0[1]['pred'],rf0[1]['label'],lut,classification_params['lc_mod'],'veg',True,os.path.join(out_dir,'veg_cms'),classification_params['model_name'])
cm_all = get_confusion_matrix(rf0[1]['pred'],rf0[1]['label'],lut,classification_params['lc_mod'],'all',True,os.path.join(out_dir,'all_cms'),classification_params['model_name'])
#cm_single = get_confusion_matrix(rf0[1]['pred'],rf0[1]['label'],lut,classification_params['lc_mod'],classification_params['lc_mod'],False,classification_params['local_model_dir'],None)

print(cm_cropNoCrop)
#print(cm_cropType)
#print(cm_veg)
#print(cm_all)
#print(cm_single)

print("Done!")

getting confusion matrix based on LC2...
getting confusion matrix based on LC_crops...
getting confusion matrix based on LC5...
getting confusion matrix based on LC25...
pred_reclass  crop  nocrop   All  correct  sumcol        UA        PA
obs_reclass                                                          
crop           727     154   881      727     830  0.825199  0.875904
nocrop         103    1734  1837     1734    1888  0.943930  0.918432
All            830    1888  2718     2718    2718  1.000000  1.000000
Done!


## view variable importance
this can be computed via Impurity or Permutation method (see sklearn docs)  by setting impmeth in rf_model
The full list is stored in the model directory for further manipulation 

In [11]:
var_imp_path = os.path.join(classification_params['model_dir'],'VarImportance_{}.csv'.format(classification_params['model_name']))
var_imp = pd.read_csv(var_imp_path, names=['var','imp'], header=None)
## view 10 most important variables:
var_imp.sort_values('imp', ascending=False).head(10)

,var,imp
110,var_poly_pred_dst,0.026115
109,var_poly_pred_ext,0.021364
132,var_kndvi_minv_wet,0.016037
133,var_kndvi_med_wet,0.014860
58,var_kndvi_avg_yr,0.013169
112,var_poly_pred_APR,0.012896
111,var_poly_pred_area,0.012297
40,var_wi_avg_yr,0.012250
68,var_kndvi_Aug_20,0.011491
67,var_kndvi_Jul_20,0.011139


In [15]:
def get_smallholder(rf_mod, one = False, half = False):
    rf_mod = pd.DataFrame(rf_mod)
    if one == True: #decide if you want to do 1ha or halfha
        smalls = rf_mod.loc[rf_mod['smalls_1ha'] == 1]
        print('There are {} 1ha smallholder samples in the holdout.'.format(smalls.shape[0]))
    elif half == True: 
        smalls = rf_mod.loc[rf_mod['smalls_halfha'] == 1]
        print('There are {} half-ha smallholder samples in the holdout.'.format(smalls.shape[0]))
    
    return smalls

def small_acc(rf_mod, one = False, half = False):
    if one == True:
        one_ha = get_smallholder(rf_mod[1], one = True) # df for 1_ha smallholder ag
        one_ha_cropNoCrop = get_confusion_matrix(one_ha['pred'], one_ha['label'],lut,classification_params['lc_mod'],'cropNoCrop',False,classification_params['model_dir'],None)
        return one_ha_cropNoCrop.head(1)["crop"]/one_ha_cropNoCrop.head(1)["All"]
    
    if half == True:
        half_ha = get_smallholder(rf_mod[1], half = True) # df for half_ha smallholder ag
        half_ha_cropNoCrop = get_confusion_matrix(half_ha['pred'], half_ha['label'],lut,classification_params['lc_mod'],'cropNoCrop',False,classification_params['model_dir'],None)
        return half_ha_cropNoCrop.head(1)["crop"]/half_ha_cropNoCrop.head(1)["All"]


In [34]:
## take the cms and find take the averages of the UAs and PAs
'''
# only run this section once to create the csvs and then comment out
feature_model = classification_params['feature_model']
sample_model = classification_params['sample_model']
model_name = classification_params['model_name']
out_dir = os.path.join(classification_params['local_dir'],'cms')

cropNoCrop_metrics = pd.DataFrame({"Model": ["{}".format(model_name)],
                             "UA": [wave(os.path.join(out_dir,'cropNoCrop_cms','{}_cropNoCrop.csv'.format(model_name)), uacc = True)],
                             "PA": [wave(os.path.join(out_dir,'cropNoCrop_cms','{}_cropNoCrop.csv'.format(model_name)), pacc = True)],
                             "1_ha": [small_acc(rf0, one = True)[0]], #create a new function that uses the smallholder rf outputs in a useful manner
                             "half_ha": [small_acc(rf0, half = True)[0]],
                             "No. of Obs.": [len(classification_params["pixdf"]["LC25_name"])]})
cropType_metrics = pd.DataFrame({"Model": ["{}".format(model_name)],
                             "UA": [wave(os.path.join(out_dir,'cropType_cms','{}_cropType.csv'.format(model_name)), uacc = True)],
                             "PA": [wave(os.path.join(out_dir,'cropType_cms','{}_cropType.csv'.format(model_name)), pacc = True)],
                             "No. of Obs.": [len(classification_params["pixdf"]["LC25_name"])]})
veg_metrics = pd.DataFrame({"Model": ["{}".format(model_name)],
                             "UA": [wave(os.path.join(out_dir,'veg_cms','{}_veg.csv'.format(model_name)), uacc = True)],
                             "PA": [wave(os.path.join(out_dir,'veg_cms','{}_veg.csv'.format(model_name)), pacc = True)],
                             "No. of Obs.": [len(classification_params["pixdf"]["LC25_name"])]})
all_metrics = pd.DataFrame({"Model": ["{}".format(model_name)],
                             "UA": [wave(os.path.join(out_dir,'all_cms','{}_all.csv'.format(model_name)), uacc = True)],
                             "PA": [wave(os.path.join(out_dir,'all_cms','{}_all.csv'.format(model_name)), pacc = True)],
                             "No. of Obs.": [len(classification_params["pixdf"]["LC25_name"])]})

print(cropNoCrop_metrics)
print(cropType_metrics)
print(veg_metrics)
print(all_metrics)

types = ["cropNoCrop", "cropType", "veg", "all"] 
matrics = [cropNoCrop_metrics, cropType_metrics, veg_metrics, all_metrics]

#saves the cms to the /testing/metrics folder
for idx, i in enumerate(types):
    matrics[idx].to_csv(os.path.join(out_dir,'metrics','{}_metrics.csv'.format(i)))
'''

There are 421 1ha smallholder samples in the holdout.
getting confusion matrix based on LC2...
There are 339 half-ha smallholder samples in the holdout.
getting confusion matrix based on LC2...
            Model        UA      PA      1_ha  half_ha  No. of Obs.
0  Max_bal400mix1  0.921591  0.9391  0.747253     0.75        12917
            Model        UA        PA  No. of Obs.
0  Max_bal400mix1  0.623538  0.764432        12917
            Model        UA       PA  No. of Obs.
0  Max_bal400mix1  0.809559  0.85365        12917
            Model        UA        PA  No. of Obs.
0  Max_bal400mix1  0.552358  0.648255        12917


In [13]:
# open the csvs and add a new row with the new data

def new_wave(cm_path, stored_path, model_name):
    stored = pd.read_csv(stored_path, index_col = 0)
    new_data = pd.DataFrame({"Model": [model_name],
                             "UA": [wave(cm_path, uacc = True)],
                             "PA": [wave(cm_path, pacc = True)],
                             "1_ha": [small_acc(rf0, one = True)[0]],
                             "half_ha": [small_acc(rf0, half = True)[0]],
                             "No. of Obs.": [len(classification_params["pixdf"]["LC25_name"])]})
    #print(stored.reset_index(drop=True))
    #print(new_data.reset_index(drop=True))
    stored = pd.concat([stored.reset_index(drop = True), new_data.reset_index(drop = True)], ignore_index = True)
    return stored


In [16]:
out_dir = os.path.join(classification_params['local_dir'],'cms')
types = ["cropNoCrop", "cropType", "veg", "all"]
mats = [cm_cropNoCrop, cm_cropType, cm_veg, cm_all]

#updates the metric csvs in /testing/metrics folder
for idx, i in enumerate(types):
    mats[idx] = new_wave(os.path.join(out_dir,'{}_cms'.format(i),'{}_{}.csv'.format(classification_params['model_name'], i)), 
                os.path.join(out_dir,'metrics','{}_metrics.csv'.format(i)),classification_params['model_name'])
    mats[idx].to_csv(os.path.join(out_dir,'metrics','{}_metrics.csv'.format(i)))
    print(mats[idx])


There are 549 1ha smallholder samples in the holdout.
getting confusion matrix based on LC2...
There are 464 half-ha smallholder samples in the holdout.
getting confusion matrix based on LC2...
            Model        UA        PA      1_ha   half_ha  No. of Obs.
0  Max_bal400mix1  0.921591  0.939100  0.747253  0.750000        12917
1  Max_bal400mix1  0.921591  0.939100  0.747253  0.750000        12917
2  Max_bal400mix2  0.921588  0.934688  0.771429  0.764045        14142
3  Max_bal400mix3  0.923043  0.931445  0.781955  0.780702        15366
There are 549 1ha smallholder samples in the holdout.
getting confusion matrix based on LC2...
There are 464 half-ha smallholder samples in the holdout.
getting confusion matrix based on LC2...
            Model        UA        PA  No. of Obs.      1_ha   half_ha
0  Max_bal400mix1  0.623538  0.764432        12917       NaN       NaN
1  Max_bal400mix1  0.623538  0.764432        12917  0.747253  0.750000
2  Max_bal400mix2  0.696597  0.773772       

In [592]:
## use this to remove certain rows from the dfs that you do not want anymore

"""
types = ["cropNoCrop", "cropType", "veg", "all"]

delrow = 0
for i in types:
    mat = pd.read_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/{}_metrics.csv".format(i), index_col = 0)
    
    mat = mat.drop([delrow])
    print(mat)
    mat.to_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/{}_metrics.csv".format(i))
"""


'\ntypes = ["cropNoCrop", "cropType", "veg", "all"]\n\ndelrow = 0\nfor i in types:\n    mat = pd.read_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/{}_metrics.csv".format(i), index_col = 0)\n    \n    mat = mat.drop([delrow])\n    print(mat)\n    mat.to_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/{}_metrics.csv".format(i))\n'

In [17]:
def overall_wave(cnc_metrics, ct_path, model_name):
    
    # iterate through rows of the CNC_metrics dataframe
    cnc = pd.read_csv(cnc_metrics, index_col = 0)
    cnc_partial = cnc[cnc["Model"] == model_name]

    # now deal with the CT matrix
    ct = pd.read_csv(ct_path, index_col = 0)
    ct_partial = [wave(ct_path, uacc = True, weights = "CT"), wave(ct_path, pacc = True, weights = "CT")]
    
    cnc_partial = [cnc_partial["UA"], cnc_partial["PA"], cnc_partial["1_ha"], cnc_partial["half_ha"]]
    #print(cnc_partial)
    #print(ct_partial)

    overall_metrics = pd.DataFrame({"Model": ["{}".format(model_name)],
                             "UA": [(2 * cnc_partial[0] + ct_partial[0])/3],
                             "PA": [(2 * cnc_partial[1] + ct_partial[1])/3],
                             "1_ha": [cnc_partial[2]],
                             "half_ha": [cnc_partial[3]],
                             "No. of Obs.": [len(classification_params["pixdf"]["LC25_name"])]})
    return overall_metrics


In [18]:
out_dir = os.path.join(classification_params['local_dir'],'cms')
print(overall_wave(os.path.join(out_dir,'metrics','cropNoCrop_metrics.csv'), 
                        os.path.join(out_dir,'cropType_cms','{}_cropType.csv'.format(classification_params['model_name'])), 
                        classification_params['model_name']))

            Model                                      UA  \
0  Max_bal400mix3  3    0.846758
Name: UA, dtype: float64   

                                       PA  \
0  3    0.871778
Name: PA, dtype: float64   

                                       1_ha  \
0  3    0.781955
Name: 1_ha, dtype: float64   

                                       half_ha  No. of Obs.  
0  3    0.780702
Name: half_ha, dtype: float64        15366  


In [42]:
## create the baseline for the new overall metrics, once again comment this code out after running it for the first time
'''
overall = overall_wave(os.path.join(out_dir,'metrics','cropNoCrop_metrics.csv'), 
             os.path.join(out_dir,'cropType_cms','{}_cropType.csv'.format(classification_params['model_name'])),
                       classification_params['model_name'])
overall.to_csv(os.path.join(out_dir,'metrics','overall_metrics.csv'))
'''

In [19]:
## add new rows to the overall_metrics csv

stored = pd.read_csv(os.path.join(out_dir,'metrics','overall_metrics.csv'), index_col = 0)
new_data = overall_wave(os.path.join(out_dir,'metrics','cropNoCrop_metrics.csv'), 
                        os.path.join(out_dir,'cropType_cms','{}_cropType.csv'.format(classification_params['model_name'])), 
                       classification_params['model_name'])
#print(stored.reset_index(drop=True))
#print(new_data.reset_index(drop=True))
stored = pd.concat([stored.reset_index(drop = True), new_data.reset_index(drop = True)], ignore_index = True)
print(stored)
stored.to_csv(os.path.join(out_dir,'metrics','overall_metrics.csv'))

            Model                                                 UA  \
0  Max_bal400mix1  0    0.812848\n1    0.812848\nName: UA, dtype:...   
1  Max_bal400mix1  0    0.812848\n1    0.812848\nName: UA, dtype:...   
2  Max_bal400mix2            2    0.844101\nName: UA, dtype: float64   
3  Max_bal400mix3             3    0.846758
Name: UA, dtype: float64   

                                                  PA  \
0  0    0.870651\n1    0.870651\nName: PA, dtype:...   
1  0    0.870651\n1    0.870651\nName: PA, dtype:...   
2            2    0.870507\nName: PA, dtype: float64   
3             3    0.871778
Name: PA, dtype: float64   

                                                1_ha  \
0  0    0.747253\n1    0.747253\nName: 1_ha, dtyp...   
1  0    0.747253\n1    0.747253\nName: 1_ha, dtyp...   
2          2    0.771429\nName: 1_ha, dtype: float64   
3           3    0.781955
Name: 1_ha, dtype: float64   

                                             half_ha  No. of Obs.  
0  0    0

In [597]:
## use this to remove certain rows from the df that you do not want anymore

"""
delrow = 16
mat = pd.read_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/overall_metrics.csv", index_col = 0)
mat = mat.drop([delrow])
print(mat)
mat.to_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/overall_metrics.csv")
"""


'\ndelrow = 16\nmat = pd.read_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/overall_metrics.csv", index_col = 0)\nmat = mat.drop([delrow])\nprint(mat)\nmat.to_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/overall_metrics.csv")\n'

## Make some tables that organize the data cleanly

In [21]:
## Organize all of the models that exclude poly_pred variables

key_word = "Max_no_pp_"
model_name = classification_params['model_name']

stored = pd.read_csv(os.path.join(out_dir,'metrics','overall_metrics.csv'), index_col = 0)
tab = []
for i in stored.iterrows():
    if i[1]["Model"] == "Max_{}".format(model_name):
        tab.append(i[1])
    if i[1]["Model"].startswith(key_word):
        tab.append(i[1])

tab = pd.DataFrame(tab)
print(tab)


Empty DataFrame
Columns: []
Index: []


In [599]:
## 




In [600]:
## some cool graphs of the data you could have compiled








## To save an html copy of this notebook with all outputs:
(these two cells should be last in notebook)

In [601]:
%%script echo skipping
### comment out above line and run this cell to print output as html

out_name = str(basic_config['country']+'6c_RandomFoest_ModelComparisons'+'_model'+str(classification_params['model_name'])+'basic_config['filter_yr'])
!jupyter nbconvert --output-dir='./Outputs' --to html --no-input --ExecutePreprocessor.store_widget_state=True --output=$out_name 6b_RandomFoest_ModelComparisons.ipynb

skipping
